In [1]:
import pyautogui
from cvzone.HandTrackingModule import HandDetector
import cv2
import math

In [2]:
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=1)
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img)
    cv2.rectangle(img, (80, 105), (580, 375), (0, 255, 0), 3)


    if hands:
        hand1 = hands[0]
        lmList1 = hand1["lmList"]
        bbox1 = hand1["bbox"]
        centerPoint1 = hand1['center']
        handType1 = hand1["type"]
        fingers1 = detector.fingersUp(hand1)

        cx1 = lmList1[8][0]   #first finger point
        cy1 = lmList1[8][1]
        cx2 = lmList1[4][0]   #thumb point
        cy2 = lmList1[4][1]
        cx = (cx1+cx2)//2
        cy = (cy1+cy2)//2
        # print(cx,cy)
        cv2.circle(img, (cx,cy),10,(255,255,0), cv2.FILLED)
        cv2.line(img,(cx1,cy1),(cx2,cy2),(0,255,0),2)
        #rectangle

        length = math.hypot(cx2 - cx1, cy2 - cy1)
        print(length)

        pyautogui.moveTo(cx * 0, cy * 4)
        if length < 100:
            pyautogui.mouseDown(cx*0,cy*4)
            #pyautogui.moveTo(cx*4,cy*4)
        if length > 10:
            pyautogui.mouseUp()
            # break

    # Display
    cv2.imshow("Image", img)
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


36.76955262170047
79.02531240052139
277.31750756127894
207.4246851269154
185.52088831180168
187.61929538296428
180.89223311131963
180.95303258028034
174.7483905505284
180.62391868188442
174.54512310574592
173.9798838946618
171.40595088852663
173.76996288196645
171.40595088852663
170.6282508847817
170.6282508847817
168.67127793433
171.40595088852663
170.6282508847817
73.10950690573696
13.0
40.024992192379
59.135437767890075
64.28063471995279
45.617978911828175
52.80151512977634
36.68787265568828
52.80151512977634
51.62363799656123
192.00520826269272
193.7446773462435
195.0717816599828
194.23954283306992
193.68531178176625
189.77091452590938
189.69449122207
190.68298298484845
187.94946129212502
185.71483516402236
185.80904176062046
185.0108104949546
185.80904176062046
185.3024554613349
181.34221791960084
178.70926109186397
177.09884245810304
178.73443988218946
195.0179478919825
92.39588735436226
88.76936408468859
24.758836806279895
21.93171219946131
17.804493814764857
13.601470508735444


KeyboardInterrupt: 

In [1]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np
import HandTrackingModule as htm
import autopy

ModuleNotFoundError: No module named 'autopy'

In [4]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):

            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        #totalFingers = fingers.count(1)

        return fingers

    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(1)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)


if __name__ == "__main__":
    main()

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


TypeError: create_int(): incompatible function arguments. The following argument types are supported:
    1. (arg0: int) -> mediapipe.python._framework_bindings.packet.Packet

Invoked with: 0.5